In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

IMDB Movie Review Dataset (cleaned)
- Originally from [here](https://ai.stanford.edu/~amaas/data/sentiment/)
- Cleaned into a csv [here](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)

In [ ]:
df = pd.read_csv('course_data/IMDB Dataset.csv')
df.head()

## Automatic Tokenization with Spacy

In [ ]:
# tool for text
import spacy

# load information about words
!python3 -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

In [ ]:
some_text = df.iloc[9]['review']
print(some_text)

# automatically tokenize the text
tokenized_text = nlp(some_text)

In [ ]:
# it's not perfect
for token in tokenized_text:
    print(token.text)

In [ ]:
idx = 5

token = tokenized_text[idx]

# lemmatization
print('Lemmatization of', token.text, 'is', token.lemma_)

# part of speech tagging
print(token.text, 'is a', token.pos_)

# is it a stop word?
print('The fact that', token.text, 'is a stop word is', token.is_stop)

In [ ]:
# sentence segmentation
for sentence in tokenized_text.sents:
    print(sentence)

- tons more fancy features!
- Let's do a simple pipeline where we ignore non-alphabetic characters

In [ ]:
import re

a_review = df.iloc[9]['review']

# remove those <br />s
a_review = a_review.replace('<br />', ' ')
print(a_review)

# remove non-alphabetic characters
a_review = re.sub("[^A-Za-z']+", ' ', a_review)
print(a_review)

In [ ]:
# disabling some fancy features of spacy for speed
nlp = spacy.load('en_core_web_sm', disable = ['ner', 'parser'])

rows = []
for idx in tqdm(range(len(df))):
    row = df.iloc[idx].copy()
    
    # first we remove numeric characters and lowercase everything
    cleaned_review = re.sub("[^A-Za-z']+", ' ', row['review'].replace('<br />', ' ')).lower()
    
    # we let spaCy tokenize and lemmatize the text for us
    tokenized_review = nlp(cleaned_review)
    cleaned_tokenized = [token.lemma_ for token in tokenized_review if ((not token.is_stop) or (' ' in token.text))]
    
    if len(cleaned_tokenized) > 1:
        row['cleaned'] = ' '.join(cleaned_tokenized)
    rows.append(row)
df_clean = pd.DataFrame(rows)
df_clean.head()
df_clean.to_csv('course_data/IMDB_cleaned.csv')

In [ ]:
df_clean = pd.read_csv('course_data/IMDB_cleaned.csv')
df_clean.head()

## Prepare for Training

In [ ]:
# count words, send infrequent to unknown

# let's get an idea of word frequency
from collections import Counter

reviews = [review.split(' ') for review in list(df_clean['cleaned'])]
word_freq = Counter([token for review in reviews for token in review]).most_common()

In [ ]:
# no surprises here
word_freq[:10]

In [ ]:
# words only seen once
word_freq[-25:]

In [ ]:
# remove words that appear infrequently
word_freq = dict(word_freq)
print(len(word_freq))
min_freq = 5
word_dict = {}

# sending all the unknowns to 0
i = 1
for word in word_freq:
    if word_freq[word] > min_freq:
        word_dict[word] = i
        i += 1
    else:
        word_dict[word] = 0

# dictionary length        
dict_length = max(word_dict.values()) + 1
dict_length

In [ ]:
# to collate the tensors into batches, sentence need to be the same size
# we could overwrite the collate function, or we could pick a max sentence size and pad

max_length = 0
for idx in tqdm(range(len(df_clean))):
    row = df_clean.iloc[idx]
    length = len(row['cleaned'].split(' '))
    if length > max_length:
        max_length = length
print(max_length)

In [ ]:
class IMDBDataset(Dataset):
    def __init__(self, df, word_dict, max_length):
        self.df = df
        self.word_dict = word_dict
        self.sent_dict = {'negative': 0, 'positive': 1}
        self.max_len = max_length
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        review = row['cleaned'].split(' ')
        x = torch.zeros(self.max_len)
        
        # get review as a list of integers
        for idx in range(len(review)):
            # we want to front pad for RNN
            x[self.max_len - len(review) + idx] = self.word_dict[review[idx]]
            
        y = torch.tensor(self.sent_dict[row['sentiment']]).float()
        
        # embedding likes long tensors
        return x.long(), y
ds = IMDBDataset(df_clean, word_dict, max_length)
next(iter(ds))

## Models

In [ ]:
# CBOW model for sentiment analysis
# train the embedding during training
class CBOW(nn.Module):
    def __init__(self, dict_length, embedding_size):
        super(CBOW, self).__init__()
        # padding index turns off gradient for unknown tokens
        self.word_emb = nn.Embedding(dict_length, embedding_size, padding_idx=0)
        self.linear = nn.Linear(embedding_size, 1)
        self.emb_size = embedding_size
        
    def forward(self, x):
        sent_length = x.shape[1]
        x = self.word_emb(x)
        sent_length = torch.count_nonzero(x, dim=1)
        x = torch.sum(x, dim=1) / sent_length
        x = self.linear(x)
        return torch.squeeze(x)

In [ ]:
dl = DataLoader(ds, batch_size=1000, shuffle=True)
x, y = next(iter(dl))

cbow_model = CBOW(dict_length, 100)
cbow_model(x).shape

In [ ]:
def one_pass(model, dataloader, optimizer, lossFun, backwards=True, print_loss=False):
    
    if backwards == True:
        model.train()
    else:
        model.eval()
    
    total_loss = 0.0
    for x, y in tqdm(dataloader):
        
        y_pred = model(x)
        loss = lossFun(y_pred, y)
        total_loss += loss.item()
        
        if backwards == True:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    avg_loss = total_loss / len(dataloader)
    
    if print_loss == True:
        print(avg_loss)
    
    return avg_loss

def one_pass_acc(model, dataloader, num_points):
    model.eval()
    total_incorrect = 0
        
    for x, y in dataloader:
        y_pred = (torch.sigmoid(model(x)) > 0.5).float()
        
        total_incorrect += torch.count_nonzero(y - y_pred).item()
        
    percent_wrong = total_incorrect / num_points
    return 1 - percent_wrong

In [ ]:
lossFun = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr = 0.01)

num_epochs = 5

for epoch in tqdm(range(num_epochs)):
    print('Epoch: ', epoch)
    
    loss = one_pass(cbow_model, dl, optimizer, lossFun)
    print('Loss: ', loss)
    
    acc = one_pass_acc(cbow_model, dl, len(ds))
    print('Accuracy: ', acc)

In [ ]:
# RNN model for sentiment analysis (read Documentation for nn.RNN!)
# train the embedding during training
class RNN(nn.Module):
    def __init__(self, dict_length, embedding_size):
        super(RNN, self).__init__()
        # padding index turns off gradient for unknown tokens
        self.word_emb = nn.Embedding(dict_length, embedding_size, padding_idx=0)
        
        # RNN doesn't care about length of sequence
        # RNN does care about the size of the word embedding
        # hidden size dictates dimension of output of RNN
        self.rnn = nn.RNN(input_size=embedding_size, hidden_size=1, batch_first=True)
        
        # PyTorch RNN outputs a sequence of same length as input
        # For many to one, we can either use the final hidden state OR
        # slap a linear layer on the output, taking in all the hidden states
        
    def forward(self, x):
        x = self.word_emb(x)
        
        # RNN layer outputs a tuple, the output and the final hidden state
        # taking the final hidden state as output
        x = self.rnn(x)[1]
    
        return torch.squeeze(x)

x, y = next(iter(dl))
rnn_model = RNN(dict_length, 100)
rnn_model(x).shape

In [ ]:
# does way better
# hidden state updates each time it sees a new word
# intuition: probably gets excited when it sees a word like bad/good and ignores the rest
lossFun = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

num_epochs = 5

for epoch in tqdm(range(num_epochs)):
    print('Epoch: ', epoch)
    
    loss = one_pass(model, dl, optimizer, lossFun)
    print('Loss: ', loss)
    
    acc = one_pass_acc(model, dl, len(ds))
    print('Accuracy: ', acc)

## Tools for Word Embeddings

In [ ]:
# gensim is a great package for word embeddings
# easy to train your own!
import gensim.downloader

# twitter embedding might be helpful for doing NLP related to social media!
print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
# let's get the glove-wiki-gigaword-100
# you can freeze the embedding for a model, finetune the embedding, or use it as a starting point for an embedding layer
glove_emb = gensim.downloader.load('glove-wiki-gigaword-100')

In [ ]:
# you can easily perform all the fancy features of word embeddings
glove_emb.most_similar('cat')

In [ ]:
# normed cat vector
cat_vec = glove_emb.get_vector('cat')
cat_vec / np.linalg.norm(cat_vec)

In [ ]:
# get the weights
weights = glove_emb.get_normed_vectors()
weights.shape

In [ ]:
# PyTorch makes it easy to load the weights
glove_emb_layer = nn.Embedding.from_pretrained(torch.tensor(weights))

In [ ]:
cat_idx = glove_emb.get_index('cat')
glove_emb_layer(torch.tensor(cat_idx))

In [ ]:
# make sure you turn off the gradients when training!
for param in glove_emb_layer.parameters():
    print(param.requires_grad)

In [ ]:
glove_emb_layer = nn.Embedding.from_pretrained(torch.tensor(weights), freeze=True)
for param in glove_emb_layer.parameters():
    print(param.requires_grad)

In [ ]:
# define a word2vec model
from gensim.models import Word2Vec

# different options for how to perform word2vec training
# check out documentation for more options related to sampling frequent vs. infrequent words
w2v_model = Word2Vec(# only consider words that show up at least a 100 times
                     min_count = 100, 
                     
                     # context window
                     window = 2,
                     
                     #size of embedding
                     vector_size = 300)
# has methods build_vocab and train